# Demo for the 2018 BHI & BSN Data Challenge

In [1]:
# Import libraries
from tableone import TableOne
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import psycopg2
import getpass
%matplotlib inline

In [53]:
# Create a database connection
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'public, mimiciii'

In [54]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

Password:········


In [55]:
# Run query and assign the results to a Pandas DataFrame
# Requires the icustay_detail view from:
# https://github.com/MIT-LCP/mimic-code/tree/master/concepts/demographics
# And the OASIS score from:
# https://github.com/MIT-LCP/mimic-code/tree/master/concepts/severityscores
query = \
"""
WITH first_icu AS (
    SELECT i.subject_id, i.hadm_id, i.icustay_id, i.gender, i.admittime admittime_hospital, 
    i.dischtime dischtime_hospital, i.los_hospital, i.age, i.admission_type, 
    i.hospital_expire_flag, i.intime intime_icu, i.outtime outtime_icu, i.los_icu
    FROM icustay_detail i
    WHERE i.hospstay_seq = 1
    AND i.icustay_seq = 1
    AND i.age >= 16
)
SELECT f.*, o.icustay_expire_flag, o.oasis, o.oasis_prob
FROM first_icu f
LEFT JOIN oasis o
ON f.icustay_id = o.icustay_id;
"""

data = pd.read_sql_query(query,con)

# Display the first few rows of the data

In [56]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'icustay_expire_flag', 'oasis', 'oasis_prob'],
      dtype='object')

In [ ]:
data.head()

# Add day of week to DataFrame

In [ ]:
data['admitday_hospital'] = data.admittime_hospital.dt.weekday_name
data['dischday_hospital'] = data.dischtime_hospital.dt.weekday_name
data['inday_icu'] = data.intime_icu.dt.weekday_name
data['outday_icu'] = data.outtime_icu.dt.weekday_name
data.head()

# Table 1 summary statistics

In [59]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'icustay_expire_flag', 'oasis', 'oasis_prob', 'admitday_hospital',
       'dischday_hospital', 'inday_icu', 'outday_icu'],
      dtype='object')

In [63]:
columns = ['gender', 'los_hospital', 'age', 'admission_type', 'hospital_expire_flag', 
           'los_icu','icustay_expire_flag', 'oasis', 'oasis_prob']

groupby = 'inday_icu'
categorical = ['gender','admission_type','hospital_expire_flag','icustay_expire_flag']

t = TableOne(data, columns=columns, categorical=categorical, groupby=groupby)
t.tableone

inday_icu=Friday inday_icu=Monday  \
variable             level                                         
n                                          6263             6097   
admission_type       ELECTIVE      1016 (16.22)     1265 (20.75)   
                     EMERGENCY     5118 (81.72)     4687 (76.87)   
                     URGENT          129 (2.06)       145 (2.38)   
age                               74.56 (53.22)    73.13 (51.37)   
gender               F              2662 (42.5)     2559 (41.97)   
                     M              3601 (57.5)     3538 (58.03)   
hospital_expire_flag 0             5576 (89.03)     5468 (89.68)   
                     1              687 (10.97)      629 (10.32)   
icustay_expire_flag  0              5768 (92.1)     5650 (92.67)   
                     1                495 (7.9)       447 (7.33)   
los_hospital                      10.23 (11.61)      9.71 (9.92)   
los_icu                             4.15 (6.19)      3.83 (5.52)   
oasis                              31.17 (9.02)     31.21 (8.77)   
oasis_prob                          0.14 (0.14)      0.14 (0.14)   

                               inday_icu=Saturday inday_icu=Sunday  \
variable             level                                           
n                                            4235             3960   
admission_type       ELECTIVE          162 (3.83)       101 (2.55)   
                     EMERGENCY       3852 (90.96)     3681 (92.95)   
                     URGENT            221 (5.22)       178 (4.49)   
age                                 73.92 (58.58)    75.26 (60.66)   
gender               F               1857 (43.85)     1736 (43.84)   
                     M               2378 (56.15)     2224 (56.16)   
hospital_expire_flag 0               3658 (86.38)     3388 (85.56)   
                     1                577 (13.62)      572 (14.44)   
icustay_expire_flag  0               3811 (89.99)      3548 (89.6)   
                     1                424 (10.01)       412 (10.4)   
los_hospital                        10.00 (10.84)     9.82 (10.70)   
los_icu                               4.42 (6.58)      4.44 (6.29)   
oasis                                31.49 (9.27)     32.07 (9.02)   
oasis_prob                            0.15 (0.15)      0.16 (0.15)   

                               inday_icu=Thursday inday_icu=Tuesday  \
variable             level                                            
n                                            5876              6141   
admission_type       ELECTIVE          999 (17.0)      1292 (21.04)   
                     EMERGENCY       4746 (80.77)       4704 (76.6)   
                     URGENT            131 (2.23)        145 (2.36)   
age                                 75.51 (55.46)     75.48 (55.22)   
gender               F                2603 (44.3)      2671 (43.49)   
                     M                3273 (55.7)      3470 (56.51)   
hospital_expire_flag 0               5202 (88.53)      5491 (89.42)   
                     1                674 (11.47)       650 (10.58)   
icustay_expire_flag  0               5399 (91.88)      5673 (92.38)   
                     1                 477 (8.12)        468 (7.62)   
los_hospital                         9.88 (10.90)      9.86 (10.58)   
los_icu                               3.96 (6.12)       3.80 (5.65)   
oasis                                31.10 (8.82)      30.90 (8.77)   
oasis_prob                            0.14 (0.14)       0.14 (0.14)   

                               inday_icu=Wednesday isnull  
variable             level                                 
n                                             5985         
admission_type       ELECTIVE         1243 (20.77)      0  
                     EMERGENCY        4600 (76.86)         
                     URGENT             142 (2.37)         
age                                  74.16 (53.88)      0  
gender               F                2636 (44.04)     

In [41]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'icustay_expire_flag', 'oasis', 'oasis_prob', 'admitday_hospital',
       'dischday_hospital', 'inday_icu', 'outday_icu'],
      dtype='object', name='variable')